<a href="https://colab.research.google.com/github/joekelly211/masfi/blob/main/7_predictions.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Imports and directories

In [ ]:
# Define base directory
# Use '/content/drive/MyDrive/' for a personal drive
# Use '/gdrive/Shareddrives/' for a shared drive (must be created first)

base_dir = "/gdrive/Shareddrives/masfi"
# base_dir = '/content/drive/MyDrive/masfi'

# Mount Google Drive
from google.colab import drive
import os
import sys
if base_dir.startswith('/gdrive/Shareddrives/'):
  drive.mount('/gdrive', force_remount=True)
elif base_dir.startswith('/content/drive/MyDrive/'):
  drive.mount('/content/drive', force_remount=True)
  os.makedirs(base_dir, exist_ok=True)
else: print("Create a base_dir beginning with '/gdrive/Shareddrives/' or '/content/drive/MyDrive/'.")

_path_to_add = os.path.realpath(base_dir)
if _path_to_add not in sys.path:
    sys.path.append(_path_to_add)

In [ ]:
# Capture outputs
%%capture
# Installs and upgrades
!apt-get install -y gdal-bin
!pip install geopandas
!pip install rasterstats
!pip install xgboost

In [ ]:
# Imports
try: import cupy # Only works on GPU runtime
except: None
import ast
from concurrent.futures import ThreadPoolExecutor
import gc
import geopandas as gpd
from google.colab import runtime
import ipywidgets as widgets
import json
import numpy as np
from numpy import random
from numpy.random import normal
from os import makedirs
from os.path import join, exists, basename
from osgeo import gdal, osr
gdal.UseExceptions()
import pickle
import pandas as pd
import psutil
from rasterstats import zonal_stats
import re
import requests
import shutil
from shutil import copyfile
from sklearn.metrics import root_mean_squared_error, r2_score
import xgboost as xgb
import warnings

In [ ]:
# Define directories
areas_dir = join(base_dir, "1_areas")
polygons_dir = join(areas_dir, "polygons")
masks_dir = join(areas_dir, "masks")
targets_dir = join(base_dir, "2_targets")
gedi_raster_final_dir = join(targets_dir, "gedi_raster_final")
targets_pkl_final_dir = join(targets_dir, "pkl_final")
feature_dir = join(base_dir, "3_features")
feature_alpha_earth_dir = join(feature_dir, "alpha_earth")
models_dir = join(base_dir, "5_models")
scenarios_dir = join(base_dir, "6_scenarios")
predictions_dir = join(base_dir, "7_predictions")
model_comparison_dir = join(predictions_dir, "model_comparison")
products_dir = join(model_comparison_dir, "products")

# Create directories
makedirs(predictions_dir, exist_ok=True)
makedirs(model_comparison_dir, exist_ok=True)
makedirs(products_dir, exist_ok=True)

In [ ]:
# Global function: export an array as a .tif
template_tif_path = join(areas_dir, "template.tif")
nodatavalue = -11111
compress = True
def export_array_as_tif(input_array, output_tif, template=template_tif_path, nodatavalue=nodatavalue, compress=compress, dtype=gdal.GDT_Float32):
    template_ds = gdal.Open(template)
    template_band = template_ds.GetRasterBand(1)
    template_dimensions, template_projection = template_ds.GetGeoTransform(), template_ds.GetProjection()
    if compress: options = ['COMPRESS=ZSTD', 'ZSTD_LEVEL=1'] # Good speed / size ratio
    else: options = []
    if input_array.dtype == 'int16': dtype = gdal.GDT_Int16
    driver = gdal.GetDriverByName("GTiff").Create(output_tif, template_band.XSize, template_band.YSize, 1, dtype, options=options)
    driver.GetRasterBand(1).WriteArray(input_array)
    driver.GetRasterBand(1).SetNoDataValue(nodatavalue)
    driver.SetGeoTransform(template_dimensions)
    driver.SetProjection(template_projection)
    template_ds = driver = None

# Global function: read raster as array
def read_raster_as_array(path):
    ds = gdal.Open(path)
    arr = ds.ReadAsArray()
    ds = None
    return arr

# Global function: Sample raster values
def sample_raster_values(pd_dataframe, raster_path, geom_x, geom_y, feature=False, n_threads=1):
    # Derive column name from filename
    raster_name = raster_path.split('/')[-1][:-4]
    if feature: raster_name = 'fea_' + raster_name
    # Load raster and extract metadata
    raster = gdal.Open(raster_path)
    band = raster.GetRasterBand(1)
    geotransform = raster.GetGeoTransform()
    raster_array = band.ReadAsArray()
    nodata = band.GetNoDataValue()
    rows, cols = raster_array.shape
    fill_value = nodata if nodata is not None else np.nan
    # Initialise output array with nodata
    sampled_values = np.full(len(geom_x), fill_value, dtype=raster_array.dtype)
    # Worker function for threaded sampling
    def sample_chunk(start, end):
        x_idx = ((geom_x[start:end] - geotransform[0]) / geotransform[1]).astype(int)
        y_idx = ((geom_y[start:end] - geotransform[3]) / geotransform[5]).astype(int)
        valid = (x_idx >= 0) & (x_idx < cols) & (y_idx >= 0) & (y_idx < rows)
        local_values = np.full(end - start, fill_value, dtype=raster_array.dtype)
        local_values[valid] = raster_array[y_idx[valid], x_idx[valid]]
        sampled_values[start:end] = local_values
    # Split points into chunks and process in parallel
    n_points = len(geom_x)
    chunk_size = (n_points + n_threads - 1) // n_threads
    chunk_ranges = [(i, min(i + chunk_size, n_points)) for i in range(0, n_points, chunk_size)]
    with ThreadPoolExecutor(max_workers=n_threads) as executor:
        executor.map(lambda r: sample_chunk(*r), chunk_ranges)
    # Assign to dataframe and release resources
    pd_dataframe[raster_name] = sampled_values
    raster = band = None

# Select model

In [ ]:
# Select a baseline model, tested and trained in advance.
# The target must have an uncertainty metric - otherwise
# skip to the next notebook '8_statistics' and use the outputs
# of the '6_scenarios' notebook.
model_exists = False
for subdir, dirs, files in os.walk(models_dir):
  for file in files:
    if file == 'model.json':
      print(f'selected_model = "{subdir.split(f"{models_dir}/",1)[1]}"')
      model_exists = True
if not model_exists:
  print("No model exists.")

In [ ]:
selected_model = "agbd_v2_1_alpha_earth_260207_194523"

# This must be True when using AlphaEarth features.
alpha_earth = False
if 'alpha_earth' in selected_model: alpha_earth = True

# Define model directories
selected_model_dir = join(models_dir,selected_model)
selected_model_json = join(selected_model_dir, "model.json")
selected_model_descr_dir = join(selected_model_dir, "model_description.json")
selected_model_dataset_path = join(selected_model_dir, f"{selected_model}.pkl")
selected_model_dataset = pd.read_pickle(selected_model_dataset_path)

# Read description for model dataset attributes
with open(join(selected_model_dir,"model_dataset_description.json")) as model_dataset_description_json:
  model_dataset_description = json.load(model_dataset_description_json)
model_dataset_name = model_dataset_description["model_dataset_name"]
number_of_columns = model_dataset_description["number_of_columns"]
number_of_rows = model_dataset_description["number_of_rows"]
id_column = model_dataset_description["id_column"]
selected_target = model_dataset_description["selected_target"]
uncertainty = model_dataset_description["uncertainty"]
covariates_renamed = model_dataset_description["covariates_renamed"]
selected_features = model_dataset_description["selected_features"] + model_dataset_description["covariates_renamed"]
categorical_features_mappings = model_dataset_description["categorical_features_mappings"]
categorical_columns = model_dataset_description["categorical_columns"]
descriptive_parameters = model_dataset_description["descriptive_parameters"]
sample_imported_dataset = model_dataset_description["sample_imported_dataset"]
sample_imported_dataset_by_percent = model_dataset_description["sample_imported_dataset_by_percent"]
sample_imported_dataset_value = model_dataset_description["sample_imported_dataset_value"]

# Reload hyperparameters
with open(selected_model_descr_dir) as model_description_json:
  model_description = json.load(model_description_json)
final_hyperparameters = ast.literal_eval(model_description["hyperparameters"])

# Remove early stopping and replace with mean n_estimators
if "early_stopping_rounds" in final_hyperparameters:
  final_hyperparameters = {k:v for k, v in final_hyperparameters.items() if k != "early_stopping_rounds"}
  final_hyperparameters["n_estimators"] = round(model_description["n_estimators mean"])

# Define directories
scenarios_model_dir = join(scenarios_dir, selected_model)
features_dir = join(scenarios_model_dir, "features")
tile_templates_dir = join(scenarios_model_dir, 'tile_templates')
feature_stacks_dir = join(scenarios_model_dir, 'feature_stacks')
predictions_model_dir = join(predictions_dir, selected_model)
model_iterations_dir = join(predictions_model_dir, "model_iterations")
scenario_iterations_dir = join(predictions_model_dir, "scenario_iterations")
tile_iteration_cache_dir = join(predictions_model_dir, "tile_iteration_cache")
tile_prediction_templates_dir = join(predictions_model_dir, "tile_prediction_templates")
tile_prediction_cache_dir = join(predictions_model_dir, "tile_prediction_cache")
scenario_predictions_dir = join(predictions_model_dir, "scenario_predictions")

# Create directories
makedirs(predictions_model_dir, exist_ok=True)
makedirs(model_iterations_dir, exist_ok=True)
makedirs(scenario_iterations_dir, exist_ok=True)
makedirs(tile_iteration_cache_dir, exist_ok=True)
makedirs(tile_prediction_templates_dir, exist_ok=True)
makedirs(tile_prediction_cache_dir, exist_ok=True)
makedirs(scenario_predictions_dir, exist_ok=True)

# Model iterations

In [ ]:
# Verify that the target is equal to the mean
print(f'mean = "{selected_target}"')

# Calculate se from columns flagged 'uncertainty'
if len(uncertainty)==0:
  print("There are no flagged uncertainty columns to calculate SE from.")
  print("Manually create the metric from the available columns.")
  for col in selected_model_dataset.columns: print(f"{col}")
else:
  for col in selected_model_dataset.columns:
    if col in uncertainty and col not in selected_target: print(f'se = "{col}"')

In [ ]:
mean = "tar_agbd"
se = "tar_agbd_se"

# GEDI L4A agbd_se represents the prediction standard error incorporating both model
# parameter uncertainty and residual variance (GEDI L4A ATBD Eq. 9, Kellner et al. 2021).
# For Monte Carlo uncertainty propagation, agbd_se directly parameterises the standard
# deviation of the prediction error distribution for each footprint.

# Set model iterations
model_iterations = 100

# Define model (y axis changes for each iteration based on mean and se arrays)
model_dataset_x = selected_model_dataset[selected_features].copy()
for col in categorical_columns:
    if col in model_dataset_x.columns:
        model_dataset_x[col] = model_dataset_x[col].astype('category')
mean_array = selected_model_dataset[mean].values
se_array = selected_model_dataset[se].values

# Detect model type from existing model or determine from target
existing_model_path = join(model_iterations_dir, "model_iteration_1.json")
if exists(existing_model_path):
    # Load existing model and detect type
    temp_booster = xgb.Booster()
    temp_booster.load_model(existing_model_path)
    model_config = json.loads(temp_booster.save_config())

    objective_name = model_config['learner']['objective']['name']
    num_class = int(model_config['learner']['learner_model_param'].get('num_class', '0'))
    classification = any(obj_type in objective_name for obj_type in ['logistic', 'softprob', 'softmax'])
    multiclass = classification and num_class > 2
else:
    # Determine from target variable characteristics
    unique_values = len(np.unique(mean_array))
    if unique_values <= 10 and all(val == int(val) for val in np.unique(mean_array)):
        classification = True
        multiclass = unique_values > 2
        num_class = unique_values if multiclass else 0
    else:
        classification = False
        multiclass = False
        num_class = 0

# Set model type
if classification:
    XGBPredictor = xgb.XGBClassifier(**final_hyperparameters)
    if multiclass: print(f"Model type: Multiclass classification ({num_class} classes)")
    else: print("Model type: Binary classification")
else:
    XGBPredictor = xgb.XGBRegressor(**final_hyperparameters)
    print("Model type: Regression")

model_params = XGBPredictor.get_params()
model_params['eval_metric'] = model_description['metric_used_for_training']
# Default fix for new XGBoost version
[model_params.pop(key, None) for key in ['n_estimators', 'enable_categorical', 'missing']]

# Progress label
model_progress_index = 0
model_progress_label = widgets.Label(f"Model iteration: {model_progress_index}/{model_iterations}")
display(model_progress_label)

for model_iteration in range(1,model_iterations+1):
  # Set model iteration filename and check if already exists
  model_iteration_filename = f"model_iteration_{model_iteration}.json"
  model_iteration_path = join(model_iterations_dir, model_iteration_filename)
  # If model iteration does not exist...
  if not exists(model_iteration_path):
    # Set the random seed based on iteration for replicability
    np.random.seed(model_iteration)
    # Set a normal distribution sample as the y for this iteration
    model_dataset_y = normal(mean_array, se_array)
    # Create DMatrix objects
    model_dtrain = xgb.DMatrix(model_dataset_x, model_dataset_y, enable_categorical=True)
    # Train the model iteration using the tested hyperparameters
    model = xgb.train(model_params,
                        model_dtrain,
                        num_boost_round=final_hyperparameters['n_estimators'],
                        verbose_eval=True)
    # Save the model iteration
    model.save_model(model_iteration_path)
  # Update progress
  model_progress_index += 1
  model_progress_label.value = f"Model iteration: {model_progress_index}/{model_iterations}"
print("All model iterations have been trained and saved.")

# Scenario iterations

In [ ]:
# Scenarios must be designed and tested using 6_scenarios.ipynb first.

# Check existing tile parameters
template_tile_list = [f for f in os.listdir(tile_templates_dir) if f.endswith('.tif') and f.startswith('template_tile')]
n_tiles = len(template_tile_list)
assert n_tiles > 0, "There are no template tiles. Run the template tiles section, even if only one is created."
if n_tiles == 1: print(f"# There is 1 template tile.\n")
if n_tiles > 1: print(f"# There are {n_tiles} template tiles.\n")

# Collect available scenarios from the feature stack tiles directory
scenario_stacks_list = []
for scenario in os.listdir(feature_stacks_dir):
    stack_files = [f for f in os.listdir(join(feature_stacks_dir, scenario)) if f.startswith('feature_stack_')]
    if len(stack_files) == n_tiles:
        scenario_stacks_list.append(scenario)

print("scenarios_to_predict = [")
for scenario in sorted(scenario_stacks_list):
  print(f'  "{scenario}",')
print("]")

In [ ]:
# There is 1 template tile.

scenarios_to_predict = [
  "2023",
  # "2024",
  # "2024_undisturbed_since_1996",
  # "2024_undisturbed_since_1997",
  # "2024_undisturbed_since_1998",
  # "2024_undisturbed_since_1999",
  # "2024_undisturbed_since_2000",
  # "2024_undisturbed_since_2001",
  # "2024_undisturbed_since_2002",
  # "2024_undisturbed_since_2003",
  # "2024_undisturbed_since_2004",
  # "2024_undisturbed_since_2005",
  # "2024_undisturbed_since_2006",
  # "2024_undisturbed_since_2007",
  # "2024_undisturbed_since_2008",
  # "2024_undisturbed_since_2009",
  # "2024_undisturbed_since_2010",
  # "2024_undisturbed_since_2011",
  # "2024_undisturbed_since_2012",
  # "2024_undisturbed_since_2013",
  # "2024_undisturbed_since_2014",
  # "2024_undisturbed_since_2015",
  # "2024_undisturbed_since_2016",
  # "2024_undisturbed_since_2017",
  # "2024_undisturbed_since_2018",
  # "2024_undisturbed_since_2019",
  # "2024_undisturbed_since_2020",
  # "2024_undisturbed_since_2021",
  # "2024_undisturbed_since_2022",
  # "2024_undisturbed_since_2023",
  # "2024_undisturbed_since_2024",
  # "2024_undisturbed_since_oldgrowth_1",
  # "2024_undisturbed_since_oldgrowth_2",
  # "2024_oldgrowth_recovery_1",
  # "2024_oldgrowth_recovery_2",
  # "2024_road_mat_daling_deforestation_2023_30m_degradation_buffer",
]

In [ ]:
# Set the number of scenario iterations. It must be <= the number of model iterations available.
scenario_iterations = 100

# Assert the number of scenario iterations is <= the number of model iterations available.
model_iterations_available = len([f for f in os.listdir(model_iterations_dir) if f.startswith('model_iteration_') and f.endswith('.json')])
assert scenario_iterations <= model_iterations_available, f"Reduce the number of scenario iterations to <= {model_iterations_available}."

# Prediction raster precision. GEDI AGBD can be set to 0, any higher is
# spurious due to the wide prediction intervals of the source data.
raster_precision = 0

# Probabilities instead of classes IF binary classification
predict_probabilities = False

# Classification threshold IF binary classification
classification_threshold = 0.5

# Detect GPU availability and set predictor type
try:
    test_array = cupy.array([1, 2, 3])
    del test_array
    predictor_type = 'gpu_predictor'
    gpu_id, use_gpu = 0, True
    print("GPU detected and accessible - using GPU to load the feature stack and to predict.")
except:
    predictor_type = 'cpu_predictor'
    gpu_id, use_gpu = -1, False
    print("GPU not accessible - using CPU prediction")
xgb.set_config(verbosity=0, use_rmm=use_gpu)

# Detect model type using first model iteration
first_model_path = join(model_iterations_dir, "model_iteration_1.json")
booster = xgb.Booster()
booster.load_model(first_model_path)
model_config = json.loads(booster.save_config())
objective_name = model_config['learner']['objective']['name']
num_class = int(model_config['learner']['learner_model_param'].get('num_class', '0'))
classification = any(obj_type in objective_name for obj_type in ['logistic', 'softprob', 'softmax'])
multiclass = classification and num_class > 2
if classification and multiclass: print(f"Model type: Multiclass classification ({num_class} classes)")
elif classification: print("Model type: Binary classification")
else: print("Model type: Regression")
booster = None

# Check existing tile parameters
template_tile_list = [f for f in os.listdir(tile_templates_dir) if f.endswith('.tif') and f.startswith('template_tile')]
n_tiles = len(template_tile_list)
if n_tiles < 1: print("There are currently no template tiles.")
else:
  template_tile = gdal.Open(join(tile_templates_dir, 'template_tile_1.tif'))
  template_tile_x = template_tile.GetRasterBand(1).XSize
  template_tile = None
  print(f"There are {n_tiles} template tiles.")

# Change template for Alpha Earth predictions, which have different spatial properties
if alpha_earth:
    if n_tiles == 1: template_base_path = join(tile_templates_dir, 'template_tile_1.tif')
    else:
        template_base_path = join(feature_alpha_earth_dir,
            next(f for f in os.listdir(feature_alpha_earth_dir) if f.endswith('.tif')))
else: template_base_path = template_tif_path

# Progress labels
n_scenarios = len(scenarios_to_predict)
scenario_progress_label = widgets.Label(value=f"Scenario progress: 0 / {n_scenarios}")
tile_progress_label = widgets.Label(value="Tile progress: -")
iteration_progress_label = widgets.Label(value="Iteration progress: -")
display(scenario_progress_label, tile_progress_label, iteration_progress_label)

# Loop through each scenario
for scenario_index, scenario in enumerate(scenarios_to_predict):
  scenario_feature_stack_dir = join(feature_stacks_dir, scenario)
  # Create scenario iterations directory
  iterations_dir = join(scenario_iterations_dir, f"{scenario}_iterations")
  makedirs(iterations_dir, exist_ok=True)

  # Check if all scenario iterations already exist
  scenario_iteration_list = []
  for model_iteration in range(1, scenario_iterations + 1):
    prediction_iteration_filename = f"{scenario}__{selected_model}_iteration_{model_iteration}.tif"
    prediction_iteration_path = join(iterations_dir, prediction_iteration_filename)
    scenario_iteration_list.append(prediction_iteration_path)
  all_scenario_iterations_exist = all(exists(p) for p in scenario_iteration_list)

  if not all_scenario_iterations_exist:
    n_stacks = len([f for f in os.listdir(scenario_feature_stack_dir) if f.startswith('feature_stack_')])

    # Single-stack prediction
    if n_stacks == 1:
      tile_progress_label.value = "Tile progress: 0 / 1"
      iteration_progress_label.value = f"Iteration progress: 0 / {scenario_iterations}"

      # Load template parameters
      template_tile_path = join(tile_templates_dir, "template_tile_1.tif")
      template_tile = gdal.Open(template_tile_path)
      template_tile_y = template_tile.GetRasterBand(1).YSize
      template_tile_x = template_tile.GetRasterBand(1).XSize
      template_tile = None
      n_pixels = template_tile_y * template_tile_x

      # Load feature stack and valid indices
      stack_path = join(scenario_feature_stack_dir, f"feature_stack_{scenario}_1.npy")
      indices_path = join(scenario_feature_stack_dir, f"valid_indices_{scenario}_1.npy")
      feature_stack = np.load(stack_path, mmap_mode='r')
      valid_indices = np.load(indices_path)
      n_valid = len(valid_indices)

      # Load to GPU if available and valid pixels exist
      if n_valid > 0 and use_gpu:
        try:
          feature_stack = cupy.asarray(feature_stack)
        except Exception as e:
          if "Memory allocation error" in str(e) or "Out of memory" in str(e):
            print("GPU memory insufficient, switching to CPU.")
            cupy.get_default_memory_pool().free_all_blocks()
            gc.collect()
            use_gpu = False
            predictor_type = 'cpu_predictor'
          else: raise

      # Predict all iterations using loaded stack
      iteration_progress_index = 0
      for model_iteration in range(1, scenario_iterations + 1):
        model_path = join(model_iterations_dir, f"model_iteration_{model_iteration}.json")
        prediction_iteration_filename = f"{scenario}__{selected_model}_iteration_{model_iteration}.tif"
        prediction_iteration_path = join(iterations_dir, prediction_iteration_filename)

        if not exists(prediction_iteration_path):
          # Load model iteration
          if classification:
            XGBPredictor = xgb.XGBClassifier()
            XGBPredictor.load_model(model_path)
            XGBPredictor.set_params(predictor=predictor_type)
            if use_gpu: XGBPredictor.set_params(device='cuda:0')
          else:
            XGBPredictor = xgb.XGBRegressor()
            XGBPredictor.load_model(model_path)
            XGBPredictor.set_params(predictor=predictor_type)
            if use_gpu: XGBPredictor.set_params(device='cuda:0')

          # Handle empty tiles
          if n_valid == 0:
            if raster_precision == 0:
              prediction_array = np.full((template_tile_y, template_tile_x), nodatavalue, dtype=np.int16)
            else:
              prediction_array = np.full((template_tile_y, template_tile_x), nodatavalue, dtype=np.float32)
          else:
            # Predict - terminate runtime if GPU prediction fails
            try:
              if classification and predict_probabilities and not multiclass:
                prediction_proba = XGBPredictor.predict_proba(feature_stack)
                prediction = prediction_proba[:, 1]
              else:
                if classification and not multiclass:
                  prediction_proba = XGBPredictor.predict_proba(feature_stack)
                  prediction = (prediction_proba[:, 1] > classification_threshold).astype(int)
                else:
                  prediction = XGBPredictor.predict(feature_stack)
                  if classification:
                    if prediction.ndim > 1 and prediction.shape[1] > 1: prediction = np.argmax(prediction, axis=1)
                    prediction = prediction.astype(int)
            except Exception as e:
              if "Memory allocation error" in str(e) or "Out of memory" in str(e):
                print("GPU memory insufficient for prediction. Terminating runtime to save compute units, restart with TPU.")
                runtime.unassign()
              else: raise

            # Reconstruct full array from valid indices (C-order)
            if raster_precision == 0:
              prediction_flat = np.full(n_pixels, nodatavalue, dtype=np.int16)
              prediction_flat[valid_indices] = np.round(prediction).astype(np.int16)
            else:
              prediction_flat = np.full(n_pixels, nodatavalue, dtype=np.float32)
              prediction_flat[valid_indices] = np.round(prediction, raster_precision)
            prediction = None
            prediction_array = prediction_flat.reshape((template_tile_y, template_tile_x), order='C')
            prediction_flat = None
          export_array_as_tif(prediction_array, prediction_iteration_path, template=template_base_path, compress=True)
          prediction_array = None

        iteration_progress_index += 1
        iteration_progress_label.value = f"Iteration progress: {iteration_progress_index} / {scenario_iterations}"
      # Clean up single-stack feature stack from memory
      feature_stack = valid_indices = None
      tile_progress_label.value = "Tile progress: 1 / 1"

    # Tiled prediction - load each stack once, predict all iterations per tile
    if n_stacks > 1:
      # Create tile cache directories for all iterations
      tile_cache_dirs = {}
      for model_iteration in range(1, scenario_iterations + 1):
        prediction_iteration_filename = f"{scenario}__{selected_model}_iteration_{model_iteration}.tif"
        tile_cache_iteration_dir = join(tile_iteration_cache_dir, prediction_iteration_filename[:-4])
        makedirs(tile_cache_iteration_dir, exist_ok=True)
        tile_cache_dirs[model_iteration] = tile_cache_iteration_dir

      # Process each tile: load stack once, predict all iterations
      for tile_count in range(1, n_stacks + 1):
        # Determine which iterations still need this tile
        iterations_needing_tile = []
        for model_iteration in range(1, scenario_iterations + 1):
          iteration_tile_path = join(tile_cache_dirs[model_iteration], f"scenario_tile_{tile_count}.tif")
          prediction_iteration_path = join(iterations_dir, f"{scenario}__{selected_model}_iteration_{model_iteration}.tif")
          if not exists(prediction_iteration_path) and not exists(iteration_tile_path):
            iterations_needing_tile.append(model_iteration)

        # Skip tile if no iterations need it
        n_iterations_for_tile = len(iterations_needing_tile)
        if n_iterations_for_tile == 0:
          iteration_progress_label.value = f"Iteration progress: {scenario_iterations} / {scenario_iterations}"
          tile_progress_label.value = f"Tile progress: {tile_count} / {n_stacks}"
          continue
        iteration_progress_label.value = f"Iteration progress: 0 / {n_iterations_for_tile}"

        # Load template tile parameters
        template_tile_path = join(tile_templates_dir, f"template_tile_{tile_count}.tif")
        template_tile = gdal.Open(template_tile_path)
        template_tile_y = template_tile.GetRasterBand(1).YSize
        template_tile_x = template_tile.GetRasterBand(1).XSize
        template_tile = None
        n_pixels = template_tile_y * template_tile_x

        # Load feature stack and valid indices once per tile
        stack_path = join(scenario_feature_stack_dir, f"feature_stack_{scenario}_{tile_count}.npy")
        indices_path = join(scenario_feature_stack_dir, f"valid_indices_{scenario}_{tile_count}.npy")
        if not exists(stack_path):
          print(f"Warning: {basename(stack_path)} not found. Skipping tile {tile_count}.")
          continue
        feature_stack = np.load(stack_path, mmap_mode='r')
        valid_indices = np.load(indices_path)
        n_valid = len(valid_indices)

        # Handle empty tiles for all needed iterations
        if n_valid == 0:
          if raster_precision == 0:
            prediction_tile = np.full((template_tile_y, template_tile_x), nodatavalue, dtype=np.int16)
          else: prediction_tile = np.full((template_tile_y, template_tile_x), nodatavalue, dtype=np.float32)
          tile_iteration_index = 0
          for model_iteration in iterations_needing_tile:
            iteration_tile_path = join(tile_cache_dirs[model_iteration], f"scenario_tile_{tile_count}.tif")
            export_array_as_tif(prediction_tile, iteration_tile_path, template=template_tile_path, compress=False)
            tile_iteration_index += 1
            iteration_progress_label.value = f"Iteration progress: {tile_iteration_index} / {n_iterations_for_tile}"
          prediction_tile = None
          tile_progress_label.value = f"Tile progress: {tile_count} / {n_stacks}"
          continue

        # Load to GPU if available
        tile_use_gpu = use_gpu
        if use_gpu:
          try: feature_stack = cupy.asarray(feature_stack)
          except Exception as e:
            if "Memory allocation error" in str(e) or "Out of memory" in str(e):
              print(f"GPU memory insufficient for tile {tile_count}, using CPU.")
              cupy.get_default_memory_pool().free_all_blocks()
              gc.collect()
              tile_use_gpu = False
            else: raise

        # Predict all needed iterations for this tile
        tile_iteration_index = 0
        for model_iteration in iterations_needing_tile:
          iteration_tile_path = join(tile_cache_dirs[model_iteration], f"scenario_tile_{tile_count}.tif")
          model_path = join(model_iterations_dir, f"model_iteration_{model_iteration}.json")
          # Load model iteration
          if classification:
            XGBPredictor = xgb.XGBClassifier()
            XGBPredictor.load_model(model_path)
            XGBPredictor.set_params(predictor='gpu_predictor' if tile_use_gpu else 'cpu_predictor')
            if tile_use_gpu: XGBPredictor.set_params(device='cuda:0')
          else:
            XGBPredictor = xgb.XGBRegressor()
            XGBPredictor.load_model(model_path)
            XGBPredictor.set_params(predictor='gpu_predictor' if tile_use_gpu else 'cpu_predictor')
            if tile_use_gpu: XGBPredictor.set_params(device='cuda:0')
          # Predict - terminate runtime if GPU prediction fails
          try:
            if classification and predict_probabilities and not multiclass:
              prediction_proba = XGBPredictor.predict_proba(feature_stack)
              prediction = prediction_proba[:, 1]
            else:
              if classification and not multiclass:
                prediction_proba = XGBPredictor.predict_proba(feature_stack)
                prediction = (prediction_proba[:, 1] > classification_threshold).astype(int)
              else:
                prediction = XGBPredictor.predict(feature_stack)
                if classification:
                  if prediction.ndim > 1 and prediction.shape[1] > 1: prediction = np.argmax(prediction, axis=1)
                  prediction = prediction.astype(int)
          except Exception as e:
            if "Memory allocation error" in str(e) or "Out of memory" in str(e):
              print("GPU memory insufficient for prediction. Terminating runtime to save compute units, restart with TPU.")
              runtime.unassign()
            else: raise

          # Reconstruct full tile from valid indices (C-order)
          if raster_precision == 0:
            prediction_flat = np.full(n_pixels, nodatavalue, dtype=np.int16)
            prediction_flat[valid_indices] = np.round(prediction).astype(np.int16)
          else:
            prediction_flat = np.full(n_pixels, nodatavalue, dtype=np.float32)
            prediction_flat[valid_indices] = np.round(prediction, raster_precision)
          prediction = None
          prediction_tile = prediction_flat.reshape((template_tile_y, template_tile_x), order='C')
          prediction_flat = None
          export_array_as_tif(prediction_tile, iteration_tile_path, template=template_tile_path, compress=False)
          prediction_tile = None

          tile_iteration_index += 1
          iteration_progress_label.value = f"Iteration progress: {tile_iteration_index} / {n_iterations_for_tile}"

        # Release tile resources
        feature_stack = valid_indices = None
        tile_progress_label.value = f"Tile progress: {tile_count} / {n_stacks}"

      # Merge tiles for each iteration
      tile_progress_label.value = "Tile progress: merging"
      for model_iteration in range(1, scenario_iterations + 1):
        prediction_iteration_filename = f"{scenario}__{selected_model}_iteration_{model_iteration}.tif"
        prediction_iteration_path = join(iterations_dir, prediction_iteration_filename)
        tile_cache_iteration_dir = tile_cache_dirs[model_iteration]
        if not exists(prediction_iteration_path):
          prediction_array = np.empty((0, template_tile_x))
          tile_files = sorted([f for f in os.listdir(tile_cache_iteration_dir) if f.endswith('.tif')],
                              key=lambda x: int(x.split('_')[-1].split('.')[0]))
          for tile_file in tile_files:
            tile_path = join(tile_cache_iteration_dir, tile_file)
            tile_array = read_raster_as_array(tile_path)
            prediction_array = np.vstack((prediction_array, tile_array))
          export_array_as_tif(prediction_array, prediction_iteration_path, template=template_base_path)
          prediction_array = None
        # Delete tile cache for this iteration
        if exists(tile_cache_iteration_dir):
          shutil.rmtree(tile_cache_iteration_dir)

        iteration_progress_label.value = f"Merge progress: {model_iteration} / {scenario_iterations}"

  else:
    tile_progress_label.value = "Tile progress: complete"
    iteration_progress_label.value = "Iteration progress: complete"

  scenario_progress_label.value = f"Scenario progress: {scenario_index + 1} / {n_scenarios}"

print("\nScenario iterations complete.")

In [ ]:
# Merge oldgrowth iteration pairs
# Old-growth scenarios exist in two versions: _1 uses land-use proxy for pre-Landsat
# undisturbed forest (e.g. PAs without history of exploitation) and removes all Landsat-derived
# disturbance. _2 removes all Landsat-derived disturbance. The merge takes the maximum mean,
# avoiding underestimation where proxy may not capture all oldgrowth characteristics of the
# entire project area. Merging at iteration level (rather than of the calculated mean)
# allows Monte Carlo uncertainty propagation when calculating AGBD percentage loss between
# the old-growth alternative scenario baseline to the 'actual' state.

# The code block needs to run uninterrupted for statistics to be calculated,
# E.g. average number of pixel contributions from each version of each scenario.

# Identify oldgrowth _1 iteration directories
oldgrowth_v1_dirs = [d for d in os.listdir(scenario_iterations_dir)
                     if 'oldgrowth' in d and d.endswith('_1_iterations')]

if not oldgrowth_v1_dirs: print("No oldgrowth version 1 iteration directories found to merge.")
else:
    n_oldgrowth = len(oldgrowth_v1_dirs)
    oldgrowth_progress_label = widgets.Label(value=f"Oldgrowth scenario progress: 0 / {n_oldgrowth}")
    iteration_merge_progress_label = widgets.Label(value="Iteration merge progress: -")
    display(oldgrowth_progress_label, iteration_merge_progress_label)

    for oldgrowth_index, v1_dir_name in enumerate(oldgrowth_v1_dirs):
        # Define directory names and paths
        v2_dir_name = v1_dir_name.replace('_1_iterations', '_2_iterations')
        merged_dir_name = v1_dir_name.replace('_1_iterations', '_iterations')
        v1_dir_path = join(scenario_iterations_dir, v1_dir_name)
        v2_dir_path = join(scenario_iterations_dir, v2_dir_name)
        merged_dir_path = join(scenario_iterations_dir, merged_dir_name)

        # Scenario name for CSV output
        scenario_name = v1_dir_name.replace('_1_iterations', '')

        # Skip if merged directory already exists with iterations
        if exists(merged_dir_path):
            existing_merged = [f for f in os.listdir(merged_dir_path) if f.endswith('.tif')]
            if existing_merged:
                iteration_merge_progress_label.value = f"Iteration merge progress: skipped (exists)"
                oldgrowth_progress_label.value = f"Oldgrowth scenario progress: {oldgrowth_index + 1} / {n_oldgrowth}"
                continue

        # Check version 2 exists
        if not exists(v2_dir_path):
            print(f"Warning: {v2_dir_name} not found, skipping merge for {v1_dir_name}")
            oldgrowth_progress_label.value = f"Oldgrowth scenario progress: {oldgrowth_index + 1} / {n_oldgrowth}"
            continue

        # Collect iteration files from both versions
        v1_files = sorted([f for f in os.listdir(v1_dir_path) if f.endswith('.tif')])
        v2_files = sorted([f for f in os.listdir(v2_dir_path) if f.endswith('.tif')])
        if len(v1_files) != len(v2_files):
            print(f"Warning: iteration count mismatch ({len(v1_files)} vs {len(v2_files)}), skipping {v1_dir_name}")
            oldgrowth_progress_label.value = f"Oldgrowth scenario progress: {oldgrowth_index + 1} / {n_oldgrowth}"
            continue
        makedirs(merged_dir_path, exist_ok=True)
        n_iterations = len(v1_files)

        # Accumulators for cross-iteration version selection and difference stats
        iteration_percentage_v1 = []
        iteration_percentage_v2 = []
        iteration_mean_absolute_difference = []
        iteration_mean_absolute_difference_v1_selected = []
        iteration_mean_absolute_difference_v2_selected = []

        # Merge each iteration pair
        for iteration_index, v1_file in enumerate(v1_files):
            # Construct merged filename (remove _1 suffix from scenario name in filename)
            merged_file = v1_file.replace('_1__', '__')
            merged_path = join(merged_dir_path, merged_file)
            if not exists(merged_path):
                # Construct v2 filename and paths
                v2_file = v1_file.replace('_1__', '_2__')
                v1_path = join(v1_dir_path, v1_file)
                v2_path = join(v2_dir_path, v2_file)
                if not exists(v2_path):
                    print(f"Warning: {v2_file} not found, skipping")
                    continue

                # Load both iterations
                v1_array = read_raster_as_array(v1_path)
                v2_array = read_raster_as_array(v2_path)

                # Take maximum value per pixel (nodata remains nodata as both versions share same mask)
                merged_array = np.maximum(v1_array, v2_array)

                # Per-pixel version selection and absolute difference stats
                valid = ~np.isnan(v1_array) & ~np.isnan(v2_array)
                absolute_difference = np.abs(v1_array - v2_array)
                v1_selected = valid & (v1_array >= v2_array)
                v2_selected = valid & (v2_array > v1_array)
                n_valid = int(valid.sum())
                n_v1 = int(v1_selected.sum())
                n_v2 = int(v2_selected.sum())
                if n_valid:
                    iteration_percentage_v1.append(100 * n_v1 / n_valid)
                    iteration_percentage_v2.append(100 * n_v2 / n_valid)
                    iteration_mean_absolute_difference.append(float(np.nanmean(absolute_difference[valid])))
                if n_v1:
                    iteration_mean_absolute_difference_v1_selected.append(float(np.nanmean(absolute_difference[v1_selected])))
                if n_v2:
                    iteration_mean_absolute_difference_v2_selected.append(float(np.nanmean(absolute_difference[v2_selected])))

                # Export merged iteration
                export_array_as_tif(merged_array, merged_path, template=v1_path, compress=True)
                merged_array = v1_array = v2_array = absolute_difference = None

            iteration_merge_progress_label.value = f"Iteration merge progress: {iteration_index + 1} / {n_iterations}"

        # Summarise and save version selection stats across iterations
        if iteration_percentage_v1:
            summary = {
                'metric': [
                    'percentage_v1_pixels', 'percentage_v2_pixels',
                    'mean_absolute_difference_all',
                    'mean_absolute_difference_v1_selected',
                    'mean_absolute_difference_v2_selected'
                ],
                'min': [
                    np.min(iteration_percentage_v1), np.min(iteration_percentage_v2),
                    np.min(iteration_mean_absolute_difference),
                    np.min(iteration_mean_absolute_difference_v1_selected) if iteration_mean_absolute_difference_v1_selected else np.nan,
                    np.min(iteration_mean_absolute_difference_v2_selected) if iteration_mean_absolute_difference_v2_selected else np.nan
                ],
                'max': [
                    np.max(iteration_percentage_v1), np.max(iteration_percentage_v2),
                    np.max(iteration_mean_absolute_difference),
                    np.max(iteration_mean_absolute_difference_v1_selected) if iteration_mean_absolute_difference_v1_selected else np.nan,
                    np.max(iteration_mean_absolute_difference_v2_selected) if iteration_mean_absolute_difference_v2_selected else np.nan
                ],
                'mean': [
                    np.mean(iteration_percentage_v1), np.mean(iteration_percentage_v2),
                    np.mean(iteration_mean_absolute_difference),
                    np.mean(iteration_mean_absolute_difference_v1_selected) if iteration_mean_absolute_difference_v1_selected else np.nan,
                    np.mean(iteration_mean_absolute_difference_v2_selected) if iteration_mean_absolute_difference_v2_selected else np.nan
                ]
            }
            stats_df = pd.DataFrame(summary).round(4)
            stats_csv_path = join(predictions_model_dir, f"{scenario_name}_version_percentage.csv")
            stats_df.to_csv(stats_csv_path, index=False)
            print(f"\n{scenario_name} version selection stats (across {len(iteration_percentage_v1)} iterations):")
            print(stats_df.to_string(index=False))
            print(f"Saved to {stats_csv_path}\n")

        oldgrowth_progress_label.value = f"Oldgrowth scenario progress: {oldgrowth_index + 1} / {n_oldgrowth}"

    print("Oldgrowth iteration merging complete.")

# Scenario predictions

In [ ]:
# Collect scenarios with iterations
# Excludes oldgrowth _1 and _2 variants since merged versions are used for statistics
scenarios_iterations_list = []
for subdir in os.listdir(scenario_iterations_dir):
    scenario_name = subdir[:-11]
    if 'oldgrowth' in scenario_name and (scenario_name.endswith('_1') or scenario_name.endswith('_2')):
        continue
    scenarios_iterations_list.append(scenario_name)

# Select scenarios to calculate mean, confidence intervals and uncertainty
print("scenarios_to_calculate = [")
for scenario in sorted(scenarios_iterations_list):
    print(f'  "{scenario}",')
print("]")

In [ ]:
scenarios_to_calculate = [
  "2023",
  # "2024",
  # "2024_undisturbed_since_1996",
  # "2024_undisturbed_since_1997",
  # "2024_undisturbed_since_1998",
  # "2024_undisturbed_since_1999",
  # "2024_undisturbed_since_2000",
  # "2024_undisturbed_since_2001",
  # "2024_undisturbed_since_2002",
  # "2024_undisturbed_since_2003",
  # "2024_undisturbed_since_2004",
  # "2024_undisturbed_since_2005",
  # "2024_undisturbed_since_2006",
  # "2024_undisturbed_since_2007",
  # "2024_undisturbed_since_2008",
  # "2024_undisturbed_since_2009",
  # "2024_undisturbed_since_2010",
  # "2024_undisturbed_since_2011",
  # "2024_undisturbed_since_2012",
  # "2024_undisturbed_since_2013",
  # "2024_undisturbed_since_2014",
  # "2024_undisturbed_since_2015",
  # "2024_undisturbed_since_2016",
  # "2024_undisturbed_since_2017",
  # "2024_undisturbed_since_2018",
  # "2024_undisturbed_since_2019",
  # "2024_undisturbed_since_2020",
  # "2024_undisturbed_since_2021",
  # "2024_undisturbed_since_2022",
  # "2024_undisturbed_since_2023",
  # "2024_undisturbed_since_2024",
  # "2024_undisturbed_since_oldgrowth",
  # "2024_oldgrowth_recovery",
  # "2024_road_mat_daling_deforestation_2023_30m_degradation_buffer",
]

In [ ]:
# Check iteration quality
# High proportion of values <= 0 may indicate corrupt iterations.
# Delete and repredict affected files if necessary.
check_iterations = False
nonpositive_threshold_percent = 1

# Check the number of prediction iterations
scenario_iterations = {}
for scenario in scenarios_to_calculate:
    iterations_dir = join(scenario_iterations_dir, f"{scenario}_iterations")
    iterations = 0
    for subdir in os.listdir(iterations_dir):
        if subdir.endswith(".tif"):
            if check_iterations:
                iteration_path = join(iterations_dir, subdir)
                iteration_array = read_raster_as_array(iteration_path)
                valid_mask = iteration_array != nodatavalue
                n_valid = np.count_nonzero(valid_mask)
                if n_valid > 0:
                    nonpositive_count = np.count_nonzero(iteration_array[valid_mask] <= 0)
                    nonpositive_percent = (nonpositive_count / n_valid) * 100
                    if nonpositive_percent > nonpositive_threshold_percent:
                        print(f"Warning: {subdir} has {nonpositive_percent:.1f}% values <= 0 in valid pixels.")
            iterations += 1
    scenario_iterations[scenario] = iterations
    print(f"There are {iterations} prediction iterations for scenario {scenario} statistics.")

In [ ]:
# Memory-aware tiling for statistics calculation.
override_n_prediction_tiles = True
n_prediction_tiles_override = 1
memory_utilisation = 0.9 # Set to <= 0.9 to ensure crashes are avoided

assert memory_utilisation > 0 and memory_utilisation <= 1, "Set memory_utilisation to a value between 0 and 1"

# Get raster dimensions from first iteration of first scenario
first_scenario = scenarios_to_calculate[0]
first_iterations_dir = join(scenario_iterations_dir, f"{first_scenario}_iterations")
first_iteration_file = next(f for f in os.listdir(first_iterations_dir) if f.endswith('.tif'))
first_iteration_path = join(first_iterations_dir, first_iteration_file)
first_ds = gdal.Open(first_iteration_path)
raster_height = first_ds.RasterYSize
raster_width = first_ds.RasterXSize
first_ds = None
print(f"Iteration raster dimensions: {raster_width} x {raster_height} pixels.")

# Max iteration count across scenarios for worst-case memory estimate
max_iterations = max(scenario_iterations.values())

# Stack per tile: n_iterations * tile_h * width * 8 bytes (float64)
# Factor of 1.1 accounts for working arrays
bytes_per_row = max_iterations * raster_width * 8 * 1.1
total_memory_needed = bytes_per_row * raster_height
print(f"RAM required for full statistics pass: ~{total_memory_needed / (1024**3):.3f} GB")
print(f"RAM currently available: {psutil.virtual_memory().available / (1024**3):.3f} GB")
n_prediction_tiles = max(1, int(np.ceil(total_memory_needed / (psutil.virtual_memory().available * memory_utilisation))))

# Calculate prediction tile dimensions (split on y-axis only)
prediction_tile_height = int(np.ceil(raster_height / n_prediction_tiles))
prediction_tile_height_remainder = raster_height % prediction_tile_height
n_prediction_tiles = max(1, len(range(0, raster_height, prediction_tile_height)))

if override_n_prediction_tiles:
    prediction_tile_height = int(np.ceil(raster_height / n_prediction_tiles_override))
    prediction_tile_height_remainder = raster_height % prediction_tile_height
    n_prediction_tiles = n_prediction_tiles_override
    print("n_prediction_tiles has been overridden.")

print(f"The statistics calculation should be divided into {n_prediction_tiles} tiles.")

# Check existing prediction template tiles
prediction_template_tile_list = [f for f in os.listdir(tile_prediction_templates_dir) if f.endswith('.tif') and f.startswith('prediction_template_tile')]
n_prediction_tiles_exist = len(prediction_template_tile_list)

if n_prediction_tiles_exist < 1: print("There are currently no prediction template tiles.")
if n_prediction_tiles_exist >= 1:
    prediction_tile_ds = gdal.Open(join(tile_prediction_templates_dir, 'prediction_template_tile_1.tif'))
    prediction_tile_height_exist = prediction_tile_ds.GetRasterBand(1).YSize
    prediction_tile_ds = None
    prediction_tile_last_ds = gdal.Open(join(tile_prediction_templates_dir, f'prediction_template_tile_{n_prediction_tiles_exist}.tif'))
    prediction_tile_height_remainder_exist = prediction_tile_last_ds.GetRasterBand(1).YSize
    prediction_tile_last_ds = None
    if n_prediction_tiles_exist == 1:
        print(f"There is a single prediction 'tile' with a height of {prediction_tile_height_exist} pixels.")
        prediction_tile_height_remainder_exist = 0
    else:
        print(f"There are {n_prediction_tiles_exist} prediction template tiles, the first {n_prediction_tiles_exist - 1} having a height of {prediction_tile_height_exist} pixels, the last {prediction_tile_height_remainder_exist} pixels.")

# Check if prediction tiles need to be changed
change_prediction_tiles = True
if override_n_prediction_tiles:
    if n_prediction_tiles == n_prediction_tiles_exist: change_prediction_tiles = False
if n_prediction_tiles == n_prediction_tiles_exist and prediction_tile_height == prediction_tile_height_exist:
    change_prediction_tiles = False

if change_prediction_tiles and n_prediction_tiles_exist >= 1:
    # Prompt before deleting cached prediction tiles
    print(f"\nPrediction tile count has changed from {n_prediction_tiles_exist} to {n_prediction_tiles}.")
    print("This will delete existing prediction template tiles and all cached prediction tiles.")
    print("Type 'OK' to proceed, or anything else to abort.")
    confirmation = input("Confirm: ").strip()
    if confirmation != "OK":
        raise RuntimeError("Aborted. Adjust override settings or re-run to keep existing tiles.")

if change_prediction_tiles:
    # Delete existing prediction template tiles
    for f in os.listdir(tile_prediction_templates_dir):
        if f.startswith('prediction_template_tile') and f.endswith('.tif'):
            os.remove(join(tile_prediction_templates_dir, f))

    # Delete cached prediction tiles for all scenarios
    for subdir in os.listdir(tile_prediction_cache_dir):
        subdir_path = join(tile_prediction_cache_dir, subdir)
        if os.path.isdir(subdir_path):
            shutil.rmtree(subdir_path)

    # Generate new prediction template tiles from first iteration raster
    tile_number = 1
    for y_start in range(0, raster_height, prediction_tile_height):
        if prediction_tile_height_remainder != 0 and tile_number == n_prediction_tiles:
            current_height = prediction_tile_height_remainder
        else: current_height = prediction_tile_height
        tiling_string = (f"gdal_translate -of GTIFF -srcwin 0, {y_start}, {raster_width}, {current_height} "
                         f"{first_iteration_path} {tile_prediction_templates_dir}/prediction_template_tile_{tile_number}.tif")
        os.system(tiling_string)
        tile_number += 1
    print("Prediction template tile creation complete.")

else: print("No changes to existing prediction tiles are required.")

In [ ]:
# Raster precision settings
mean_precision = 2
ci_precision = 2
uncertainty_precision = 2

# Quantify scenario predictions using Monte Carlo simulation with percentile-based
# confidence intervals (IPCC Approach 2) and percentage uncertainty.

# Each model iteration sampled target values from normal distributions using GEDI L4A
# agbd (mean) and agbd_se (standard deviation). The resulting collection of respective
# prediction iterations represent the distribution of possible AGBD values
# given measurement uncertainty.

# Confidence intervals derive from percentiles of the empirical (actual rather
# than theoretical) distribution: CI half-width = (P_97.5 - P_2.5) / 2
# Percentiles directly characterise the distribution without assuming normality,
# are robust to outliers, and handle bounded distributions (AGBD >= 0).

# Percentage uncertainty = CI half-width / mean * 100
# This represents precision of estimates given input uncertainty, not total predictive
# accuracy (Liang et al. 2023, p.6).

# References:
# IPCC (2006) Guidelines Vol.1 Ch.3: Uncertainties, Section 3.2.3.2
# IPCC (2019) Refinement Vol.1 Ch.3: Uncertainties, Section 3.2.3.2
# Liang et al. (2023) Remote Sensing of Environment 284:113367

# Check prediction template tiles
prediction_template_tile_list = [f for f in os.listdir(tile_prediction_templates_dir) if f.endswith('.tif') and f.startswith('prediction_template_tile')]
n_prediction_tiles = len(prediction_template_tile_list)
assert n_prediction_tiles > 0, "There are no prediction template tiles. Run the previous section."

# Prediction template tile dimensions
prediction_tile_1_ds = gdal.Open(join(tile_prediction_templates_dir, 'prediction_template_tile_1.tif'))
prediction_tile_width = prediction_tile_1_ds.GetRasterBand(1).XSize
prediction_tile_1_ds = None
print(f"There are {n_prediction_tiles} prediction template tiles.")

# Progress labels
scenario_progress_index = 0
scenario_progress_label = widgets.Label(f"Scenario predictions progress: {scenario_progress_index}/{len(scenarios_to_calculate)}")
tile_progress_label = widgets.Label(value="Prediction tile progress: -")
iteration_progress_label = widgets.Label(value="Iteration loading progress: -")
display(scenario_progress_label, tile_progress_label, iteration_progress_label)

# Loop through the scenarios
for scenario, iteration_count in scenario_iterations.items():
    base_filename = f"{scenario}__{selected_model}"
    iterations_dir = join(scenario_iterations_dir, f"{scenario}_iterations")

    # Define output paths
    mean_filename = f"mean__{base_filename}.tif"
    mean_path = join(scenario_predictions_dir, mean_filename)
    ci_filename = f"ci95__{base_filename}.tif"
    ci_path = join(scenario_predictions_dir, ci_filename)
    uncertainty_filename = f"uncertainty__{base_filename}.tif"
    uncertainty_path = join(scenario_predictions_dir, uncertainty_filename)

    # If any output does not exist, recalculate all
    if not exists(mean_path) or not exists(ci_path) or not exists(uncertainty_path):

        # Create scenario cache directory
        scenario_cache_dir = join(tile_prediction_cache_dir, scenario)
        makedirs(scenario_cache_dir, exist_ok=True)

        # Collect iteration file paths
        iteration_paths = [join(iterations_dir, f) for f in os.listdir(iterations_dir) if f.endswith(".tif")]
        n_iterations = len(iteration_paths)

        # Percentile interpolation indices (constant across tiles)
        n = n_iterations
        lower_index = 0.025 * (n - 1)
        upper_index = 0.975 * (n - 1)
        lower_floor = int(np.floor(lower_index))
        upper_floor = int(np.floor(upper_index))
        lower_fraction = lower_index - lower_floor
        upper_fraction = upper_index - upper_floor

        # Open all iteration datasets once (read strips per tile)
        iteration_progress_label.value = f"Opening {n_iterations} iteration datasets"
        iteration_datasets = [gdal.Open(p) for p in iteration_paths]

        # Process each prediction tile
        y_start = 0
        for tile_idx in range(1, n_prediction_tiles + 1):
            tile_progress_label.value = f"Prediction tile progress: {tile_idx} / {n_prediction_tiles}"

            # Cached tile output paths
            mean_tile_path = join(scenario_cache_dir, f"mean_{tile_idx}.tif")
            ci_tile_path = join(scenario_cache_dir, f"ci95_{tile_idx}.tif")
            uncertainty_tile_path = join(scenario_cache_dir, f"uncertainty_{tile_idx}.tif")

            # Get tile dimensions from prediction template
            prediction_template_path = join(tile_prediction_templates_dir, f"prediction_template_tile_{tile_idx}.tif")
            prediction_template_ds = gdal.Open(prediction_template_path)
            current_tile_height = prediction_template_ds.GetRasterBand(1).YSize
            current_tile_width = prediction_template_ds.GetRasterBand(1).XSize
            prediction_template_ds = None

            # Skip tile if all three cached outputs exist
            if exists(mean_tile_path) and exists(ci_tile_path) and exists(uncertainty_tile_path):
                iteration_progress_label.value = f"Tile {tile_idx}: skipped (cached)"
                y_start += current_tile_height
                continue

            # Load strip from each iteration into stack
            tile_stack = np.empty((n_iterations, current_tile_height, current_tile_width), dtype=np.float64)
            for iter_idx, ds in enumerate(iteration_datasets):
                band = ds.GetRasterBand(1)
                tile_stack[iter_idx] = band.ReadAsArray(0, y_start, current_tile_width, current_tile_height)
                iteration_progress_label.value = f"Tile {tile_idx}: loading iteration {iter_idx + 1} / {n_iterations}"

            # Valid mask from first iteration (identical across all iterations)
            valid_mask = tile_stack[0] != nodatavalue

            # Calculate mean across iterations
            mean_array = np.mean(tile_stack, axis=0)

            # Sort in-place, index at percentile positions. Faster than np.nanpercentile.
            # Nodata values (-11111) sort to bottom; excluded by valid_mask.
            tile_stack.sort(axis=0)
            sorted_stack = tile_stack
            tile_stack = None

            # Interpolate between adjacent sorted values
            ci_lower = (sorted_stack[lower_floor] * (1 - lower_fraction) +
                        sorted_stack[min(lower_floor + 1, n - 1)] * lower_fraction)
            ci_upper = (sorted_stack[upper_floor] * (1 - upper_fraction) +
                        sorted_stack[min(upper_floor + 1, n - 1)] * upper_fraction)
            sorted_stack = None

            # CI half-width
            ci_halfwidth = (ci_upper - ci_lower) / 2

            # Relative uncertainty as percentage
            percentage_uncertainty = np.zeros_like(mean_array, dtype=np.float64)
            nonzero_mean_mask = valid_mask & (mean_array != 0)
            percentage_uncertainty[nonzero_mean_mask] = (ci_halfwidth[nonzero_mean_mask] / np.abs(mean_array[nonzero_mean_mask])) * 100

            # Round and apply nodata mask
            if mean_precision == 0:
                mean_rounded = np.where(valid_mask, np.round(mean_array, mean_precision).astype(np.int16), nodatavalue).astype(np.int16)
            else:
                mean_rounded = np.where(valid_mask, np.round(mean_array, mean_precision), nodatavalue)
            if ci_precision == 0:
                ci_rounded = np.where(valid_mask, np.round(ci_halfwidth, ci_precision).astype(np.int16), nodatavalue).astype(np.int16)
            else:
                ci_rounded = np.where(valid_mask, np.round(ci_halfwidth, ci_precision), nodatavalue)
            if uncertainty_precision == 0:
                uncertainty_rounded = np.where(valid_mask, np.round(percentage_uncertainty, uncertainty_precision).astype(np.int16), nodatavalue).astype(np.int16)
            else:
                uncertainty_rounded = np.where(valid_mask, np.round(percentage_uncertainty, uncertainty_precision), nodatavalue)

            # Export mean, CI and uncertainty tile rasters to cache
            export_array_as_tif(mean_rounded, mean_tile_path, template=prediction_template_path, compress=True)
            export_array_as_tif(ci_rounded, ci_tile_path, template=prediction_template_path, compress=True)
            export_array_as_tif(uncertainty_rounded, uncertainty_tile_path, template=prediction_template_path, compress=True)
            mean_rounded = ci_rounded = uncertainty_rounded = None
            mean_array = ci_halfwidth = percentage_uncertainty = None

            # Update y offset
            y_start += current_tile_height

        # Close iteration datasets
        iteration_datasets = None

        # Merge cached tiles into final rasters
        tile_progress_label.value = "Prediction tile progress: merging"
        for output_path, prefix in [(mean_path, "mean"), (ci_path, "ci95"), (uncertainty_path, "uncertainty")]:
            merged_array = np.empty((0, prediction_tile_width))
            for tile_idx in range(1, n_prediction_tiles + 1):
                tile_path = join(scenario_cache_dir, f"{prefix}_{tile_idx}.tif")
                tile_array = read_raster_as_array(tile_path)
                merged_array = np.vstack((merged_array, tile_array))
            export_array_as_tif(merged_array, output_path, template=first_iteration_path)
            merged_array = None

        # Delete scenario cache directory
        shutil.rmtree(scenario_cache_dir)

    else:
        tile_progress_label.value = "Prediction tile progress: skipped (exists)"
        iteration_progress_label.value = "Iteration loading progress: skipped (exists)"
        print(f"{mean_filename}, {ci_filename} and {uncertainty_filename} already exist.")

    scenario_progress_index += 1
    scenario_progress_label.value = f"Scenario predictions progress: {scenario_progress_index}/{len(scenarios_to_calculate)}"

print("\nStatistics calculations complete.")

# Model comparison

In [ ]:
# GEDI raster patterns
l4d_prefix, l4d_suffix = "GEDI04_D_V2_original_epsg32648_UTM", "_agbd.tif"
l4b_filename = "GEDI04_B_V2_1_reprojected_epsg4326_WGS84_R01000M.tif"
veg_grid_filename = "GEDI_vegetation_grid_original_epsg6933_EASE-Grid_agbd-a0_20190417_20230316.tif"

# Forest mask year 2023 used for all products, as this was the final GEDI data collection year
gedi_mask_year = 2023

# Template for clipping
template_gdf = gpd.read_file(join(polygons_dir, "template.gpkg"))

# Locate GEDI product rasters
gedi_products = {'L4D': [], 'L4B': None, 'vegetation_grid': None}
gedi04d_dir = join(gedi_raster_final_dir, "GEDI04_D_V2")
gedi04b_dir = join(gedi_raster_final_dir, "GEDI04_B_V2_1")
veg_dir = join(gedi_raster_final_dir, "GEDI_vegetation_grid")

if exists(gedi04d_dir):
    gedi_products['L4D'] = [join(gedi04d_dir, f) for f in os.listdir(gedi04d_dir) if f.startswith(l4d_prefix) and f.endswith(l4d_suffix)]
print(f"# L4D grid: {len(gedi_products['L4D'])} UTM tiles.")

if exists(gedi04b_dir) and exists(join(gedi04b_dir, l4b_filename)):
    gedi_products['L4B'] = join(gedi04b_dir, l4b_filename)
print(f"# L4B grid: {'found' if gedi_products['L4B'] else 'not found'}")

if exists(veg_dir) and exists(join(veg_dir, veg_grid_filename)):
    gedi_products['vegetation_grid'] = join(veg_dir, veg_grid_filename)
print(f"# Vegetation grid (AGBD): {'found' if gedi_products['vegetation_grid'] else 'not found'}")

# Copy and mask GEDI rasters to forest extent
print("\nCopying and masking GEDI rasters")

# Mask raster to forest extent, resampling mask if needed
def apply_forest_mask(raster_path, mask_path, output_path):
    if exists(output_path):
        print(f"  Exists: {os.path.basename(output_path)}")
        return
    raster_ds = gdal.Open(raster_path)
    raster_proj = raster_ds.GetProjection()
    raster_gt = raster_ds.GetGeoTransform()
    raster_xsize, raster_ysize = raster_ds.RasterXSize, raster_ds.RasterYSize
    raster_bounds = [raster_gt[0], raster_gt[3] + raster_gt[5] * raster_ysize, raster_gt[0] + raster_gt[1] * raster_xsize, raster_gt[3]]
    mask_resampled = gdal.Warp('', mask_path, options=gdal.WarpOptions(format='MEM', dstSRS=raster_proj, outputBounds=raster_bounds, width=raster_xsize, height=raster_ysize, resampleAlg='near', dstNodata=nodatavalue))
    mask_array = mask_resampled.GetRasterBand(1).ReadAsArray()
    raster_array = raster_ds.GetRasterBand(1).ReadAsArray()
    # convert original data gaps to -1 (non-forest becomes nodatavalue)
    raster_array = np.where(raster_array == nodatavalue, -1, raster_array)
    raster_array = np.where(mask_array == 1, raster_array, nodatavalue)
    driver = gdal.GetDriverByName('GTiff')
    out_ds = driver.Create(output_path, raster_xsize, raster_ysize, 1, raster_ds.GetRasterBand(1).DataType, options=['COMPRESS=ZSTD', 'ZSTD_LEVEL=1'])
    out_ds.SetGeoTransform(raster_gt)
    out_ds.SetProjection(raster_proj)
    out_band = out_ds.GetRasterBand(1)
    out_band.WriteArray(raster_array)
    out_band.SetNoDataValue(nodatavalue)
    out_ds = raster_ds = mask_resampled = None
    print(f"  Created: {os.path.basename(output_path)}")

gedi_mask_path = join(masks_dir, f"mask_forest_{gedi_mask_year}.tif")

# Mask L4D rasters
l4d_masked = []
if gedi_products['L4D']:
    print("L4D rasters:")
    for raster_path in gedi_products['L4D']:
        filename = os.path.basename(raster_path).replace('_original', '').replace('.tif', f'_masked_{gedi_mask_year}.tif')
        output_path = join(products_dir, filename)
        apply_forest_mask(raster_path, gedi_mask_path, output_path)
        l4d_masked.append(output_path)

# Mask L4B raster
l4b_masked = None
if gedi_products['L4B']:
    print("L4B raster:")
    filename = os.path.basename(gedi_products['L4B']).replace('_original', '').replace('.tif', f'_masked_{gedi_mask_year}.tif')
    output_path = join(products_dir, filename)
    apply_forest_mask(gedi_products['L4B'], gedi_mask_path, output_path)
    l4b_masked = output_path

# Mask vegetation_grid raster
veg_masked = None
if gedi_products['vegetation_grid']:
    print("Vegetation grid raster:")
    filename = os.path.basename(gedi_products['vegetation_grid']).replace('_original', '').replace('.tif', f'_masked_{gedi_mask_year}.tif')
    output_path = join(products_dir, filename)
    apply_forest_mask(gedi_products['vegetation_grid'], gedi_mask_path, output_path)
    veg_masked = output_path

print(f"\nMasked rasters: L4D={len(l4d_masked)}, L4B={'1' if l4b_masked else '0'}, Vegetation grid={'1' if veg_masked else '0'}")

In [ ]:
# L4D validation metrics URL
l4d_gpkg_url = "https://data.ornldaac.earthdata.nasa.gov/public/gedi/GEDI_L4D_Imputed_Waveforms/comp/GEDI_L4D_20190418_20230316_validation_metrics.gpkg"

# Forest cover threshold for tile inclusion
l4d_forest_threshold = 0.95

# Forest mask year: 2023 used for L4D
gedi_mask_year = 2023

gpkg_raw = join(products_dir, "GEDI_L4D_validation_metrics_raw.gpkg")
gpkg_selected = join(products_dir, "GEDI_L4D_validation_metrics_selected.gpkg")
l4d_accuracy_path = join(model_comparison_dir, "l4d_tile_accuracy.csv")

# Columns required for pooled accuracy calculation
l4d_cols = ['rmse_agbd', 'agbd_mean_valid', 'valid_data_size_agbd', 'geometry']

# Download validation gpkg
if not exists(gpkg_raw):
    print("Downloading L4D validation gpkg...")
    response = requests.get(l4d_gpkg_url, stream=True)
    response.raise_for_status()
    with open(gpkg_raw, 'wb') as f:
        for chunk in response.iter_content(chunk_size=8192): f.write(chunk)
    print("  Download complete")
else: print("L4D validation gpkg exists")

# Select tiles fully within template extent
if not exists(gpkg_selected):
    print("Selecting tiles within template extent...")
    tiles_gdf = gpd.read_file(gpkg_raw)[l4d_cols].to_crs(template_gdf.crs)
    template_union = template_gdf.union_all()
    tiles_selected = tiles_gdf[tiles_gdf.within(template_union)]
    tiles_selected.to_file(gpkg_selected, driver="GPKG")
else: print("Selected gpkg exists")

# Calculate pooled accuracy for tiles meeting forest threshold
tiles_gdf = gpd.read_file(gpkg_selected)
print(f"  Tiles within template extent: {len(tiles_gdf)}")
rmse_col, mean_col, n_col = "rmse_agbd", "agbd_mean_valid", "valid_data_size_agbd"
gedi_mask_path = join(masks_dir, f"mask_forest_{gedi_mask_year}.tif")

# Create temporary mask with nodata converted to 0 (for zonal histogram)
mask_ds = gdal.Open(gedi_mask_path)
mask_array = mask_ds.GetRasterBand(1).ReadAsArray()
mask_array = np.where(mask_array == nodatavalue, 0, mask_array)
temp_mask_path = join(products_dir, "temp_forest_mask.tif")
export_array_as_tif(mask_array, temp_mask_path, template=gedi_mask_path)

# Calculate forest fraction per tile using zonal histogram
stats = zonal_stats(tiles_gdf.to_crs(mask_ds.GetProjection()), temp_mask_path, categorical=True)
forest_fractions = [s.get(1, 0) / sum(s.values()) if s else 0 for s in stats]
tiles_gdf['forest_fraction'] = forest_fractions
mask_ds = None
os.remove(temp_mask_path)

# Filter by forest threshold and valid metrics
tiles_valid = tiles_gdf[tiles_gdf['forest_fraction'] >= l4d_forest_threshold].dropna(subset=[rmse_col, mean_col, n_col])
print(f"  Tiles >={l4d_forest_threshold*100:.0f}% forest: {len(tiles_valid)}")

n = tiles_valid[n_col].values
rmse_vals = tiles_valid[rmse_col].values
mean_vals = tiles_valid[mean_col].values

# Pooled metrics
pooled_rmse = np.sqrt(np.sum(n * rmse_vals**2) / np.sum(n))
pooled_mean = np.sum(n * mean_vals) / np.sum(n)
pooled_rrmse = (pooled_rmse / pooled_mean) * 100
print(f"  RMSE={pooled_rmse:.4f}, rRMSE={pooled_rrmse:.2f}%, mean={pooled_mean:.2f}")

# Save results
l4d_accuracy = {'source': f'L4D grid (tiles >={l4d_forest_threshold*100:.0f}% forest)', 'rmse': pooled_rmse, 'rrmse': pooled_rrmse, 'n_tiles': len(tiles_valid), 'n_validation_points': int(np.sum(n)), 'forest_threshold': l4d_forest_threshold}
pd.DataFrame([l4d_accuracy]).to_csv(l4d_accuracy_path, index=False)
print(f"  Saved: {l4d_accuracy_path}")

In [ ]:
# Find model predictions
# scenarios_dir: single prediction ({year}__{model}.tif)
# predictions_dir: Monte Carlo mean (mean__{year}__{model}.tif)
model_predictions = {}
for source, pattern in [(scenarios_dir, r'^(\d{4})__'), (predictions_dir, r'^mean__(\d{4})__')]:
    if not exists(source): continue
    source_key = 'scenarios_dir' if source == scenarios_dir else 'predictions_dir'
    for model_name in os.listdir(source):
        pred_dir = join(source, model_name, 'scenario_predictions')
        if not exists(pred_dir): continue
        years = set()
        for f in os.listdir(pred_dir):
            match = re.match(pattern, f)
            if match: years.add(match.group(1))
        if years:
            if model_name not in model_predictions: model_predictions[model_name] = {}
            model_predictions[model_name][source_key] = sorted(years)

# Print selectable dictionary
print("\n# Model predictions available:")
print("selected_predictions = {")
for model, categories in model_predictions.items():
    print(f"    '{model}': {{")
    for cat, years in categories.items():
        print(f"        '{cat}': [")
        for y in years:
            print(f"            '{y}',")
        print(f"        ],")
    print(f"    }},")
print("}")

In [ ]:

# Model predictions available:
selected_predictions = {
    # 'gedi_elevation_v2_1_260202_123700': {
    #     'scenarios_dir': [
    #         '2015',
    #     ],
    # },
    'agbd_v2_1_260206_133525': {
        # 'scenarios_dir': [
        #     '2023',
        #     '2024',
        # ],
        'predictions_dir': [
            '2023',
            # '2024',
        ],
    },
    'agbd_v2_1_alpha_earth_260207_194523': {
        'predictions_dir': [
            '2023',
        ],
    },
}

In [ ]:
# Select the original GEDI 4A .pkl generated in '2_variates.ipynb'
print("Available datasets")
for f in os.listdir(targets_pkl_final_dir):
    if f.endswith(".pkl"):
        print(f'original_gedi4a_dataset = "{f}"')

In [ ]:
original_gedi4a_dataset = "GEDI04_A_V2_1.pkl"

# Load footprints from the original GEDI L4A dataset
# Accuracy comparison at sensitivity >= 0.98
# Only 2023 and 2024 footprints used,
# both have equal potential for temporal misalignment with end-of-year predictions.
original_gedi4a_df = pd.read_pickle(join(targets_pkl_final_dir, original_gedi4a_dataset))
accuracy_df = original_gedi4a_df[['geometry', 'timestamp', 'agbd', 'sensitivity']].copy()
accuracy_df['year'] = pd.to_datetime(accuracy_df['timestamp']).dt.year
accuracy_df = accuracy_df[(accuracy_df['sensitivity'] >= 0.98) & (accuracy_df['year'].isin([2023, 2024]))]

category_labels = {'scenarios_dir': 'single prediction', 'predictions_dir': 'prediction mean'}
results = []

# Build list of sources to evaluate
sources = [
    ('L4D grid', l4d_masked),
    ('L4B grid', [l4b_masked] if l4b_masked else []),
    ('Veg grid', [veg_masked] if veg_masked else []),
]
for model_name, categories in selected_predictions.items():
    for category, years in categories.items():
        pred_dir = join(scenarios_dir if category == 'scenarios_dir' else predictions_dir, model_name, 'scenario_predictions')
        for year in years:
            raster_path = join(pred_dir, f"{year}__{model_name}.tif" if category == 'scenarios_dir' else f"mean__{year}__{model_name}.tif")
            if exists(raster_path):
                sources.append((f"{model_name} ({category_labels[category]} {year})", [raster_path]))

# Evaluate each source
for label, raster_paths in sources:
    if not raster_paths: continue

    # Sample rasters
    samples = np.full(len(accuracy_df), nodatavalue, dtype=float)
    for raster_path in raster_paths:
        raster_ds = gdal.Open(raster_path)
        srs = osr.SpatialReference()
        srs.ImportFromWkt(raster_ds.GetProjection())
        epsg = int(srs.GetAuthorityCode(None))
        raster_ds = None
        gdf = gpd.GeoDataFrame(accuracy_df, geometry='geometry', crs='EPSG:4326').to_crs(epsg=epsg)
        temp_df = pd.DataFrame(index=accuracy_df.index)
        sample_raster_values(temp_df, raster_path, gdf.geometry.x.values, gdf.geometry.y.values)
        values = temp_df.iloc[:, 0].values
        valid_update = (values >= 0) & (samples < 0)
        samples[valid_update] = values[valid_update]

    # Calculate metrics
    valid_mask = samples >= 0
    if valid_mask.sum() == 0: continue
    obs, pred = accuracy_df['agbd'].values[valid_mask].astype('float32'), samples[valid_mask].astype('float32')
    rmse = root_mean_squared_error(obs, pred)
    results.append({'source': label, 'r2': r2_score(obs, pred), 'me': np.mean(obs - pred), 'rmse': rmse, 'rrmse': (rmse / np.mean(obs)) * 100, 'n_points': len(obs)})
    print(f"{label}: {len(obs)} points")

# Model cross-validation metrics
for model_name in selected_predictions.keys():
    model_desc_path = join(models_dir, model_name, "model_description.json")
    if not exists(model_desc_path): continue
    with open(model_desc_path) as f:
        model_desc = json.load(f)
    results.append({
        'source': f'{model_name} (cross-validation)',
        'r2': model_desc.get('score_validation (r2) mean', np.nan), 'me': model_desc.get('score_validation (me) mean', np.nan),
        'rmse': model_desc.get('score_validation (rmse) mean', np.nan), 'rrmse': model_desc.get('score_validation (rrmse) mean', np.nan), 'n_points': np.nan
    })
print("Cross-validation: completed")

# Build comparison table
comparison_df = pd.DataFrame(results)
col_order = ['source', 'r2', 'me', 'rmse', 'rrmse', 'n_points']
comparison_df = comparison_df[[c for c in col_order if c in comparison_df.columns]]
comparison_path = join(model_comparison_dir, "accuracy_comparison.csv")
comparison_df.to_csv(comparison_path, index=False)
print(f"\nComparison saved: {comparison_path}")
print(comparison_df.to_string(index=False))

# Disconnect runtime

In [ ]:
# Useful for stopping background execution upon completion
runtime.unassign()